# Request OSHA Enforcement Data for Springfield, IL 2018-2022
12/4/23 RR<br>
This notebook requests OSHA Enforcement Data about inspections and violations from 2018 to 2022.<br>
<br>
12/6/23 addendum<br>
I chose to access data via API request for a few reasons:<br>
<ul>
    <li>Most of OSHA's downloadable CSV files were too large to upload to Google SheetsI don't have an Excel license
<li>OSHA's files (e.g. inspections) are split into multiple files, which would need to be combined before searching
    <li>API request allows for filtering data upfront to retrieve a smaller file with only the needed records and columns needed
    </ul>

### Initial Findings
187 inspections were completed in Springfield from 2018 to 2022<br>
294 violations were recorded related to these inspections

### Next Steps for improving this notebook
The Department of Labor site's limit of 200 records per request and 200 requests per hour was sufficient for this particular analysis. To expand this analysis to queries resulting in larger datasets, I could take of the following approaches.
<ul>
    <li>After each API request, determine an "offset" based on the record number (RNUM), then continue subsequent requests until the entire OSHA table is reviewed.
        <li>Download all data (e.g. multiple inspections.csv files) and merge them together into one large file.
    </ul>

# 1) Review Data Documentation

<strong>OSHA Enforcement Data</strong><br>
https://enforcedata.dol.gov/views/data_summary.php<br>

<strong>Data Dictionary</strong><br>
https://developer.dol.gov/health-and-safety/dol-osha-enforcement/#osha_inspection<br>

<strong>API Developer Guide</strong><br>
https://developer.dol.gov/accessing-the-apis-using-http-requests/#apiv2

sample request:
<blockquote>curl -H "X-API-KEY: YOUR_API_KEY" https://data.dol.gov/get/ALIAS/filtername1/filtervalue1/filtername2/filtervalue2

<strong>Data Architecture</strong>
"Inspections" is the parent table in the database, and "violations" key to these inspections through the unique identifier "ACTIVITY_NR."

# 2) Request and Review OSHA Inspections Data

I needed to register with the Department of Labor to get an API key at<br>
https://devtools.dol.gov/developer/Account/Register

In [1]:
my_key = {"X-API-KEY": "496af341-1976-48ab-91e7-19bdfccfe825"}

In [2]:
# import necessary Python libraries

import pandas as pd
import requests

### Request Inspection Data

In [3]:
url = "https://data.dol.gov/get/inspection/date_column/OPEN_DATE/start_date/01-JAN-18/end_date/31-DEC-22/filter_column/SITE_STATE=IL:SITE_CITY=SPRINGFIELD"
response = requests.get(url, headers=my_key)
data = response.json()
response

<Response [200]>

In [4]:
#look at first record
data[0]

{'ACTIVITY_NR': '342934890',
 'REPORTING_ID': '0524500',
 'STATE_FLAG': '',
 'ESTAB_NAME': 'SPRINGFIELD FARM & HOME SUPPLY, INC.',
 'SITE_ADDRESS': '2804 N. DIRKSEN PKWY',
 'SITE_CITY': 'SPRINGFIELD',
 'SITE_STATE': 'IL',
 'SITE_ZIP': '62702',
 'OWNER_TYPE': 'A',
 'OWNER_CODE': '',
 'ADV_NOTICE': 'N',
 'SAFETY_HLTH': 'S',
 'SIC_CODE': '',
 'NAICS_CODE': '452990',
 'INSP_TYPE': 'B',
 'INSP_SCOPE': 'B',
 'WHY_NO_INSP': 'I',
 'UNION_STATUS': 'B',
 'SAFETY_MANUF': '',
 'SAFETY_CONST': '',
 'SAFETY_MARIT': '',
 'HEALTH_MANUF': '',
 'HEALTH_CONST': '',
 'HEALTH_MARIT': '',
 'MIGRANT': '',
 'MAIL_STREET': '2804 N. DIRKSEN PKWY',
 'MAIL_CITY': 'SPRINGFIELD',
 'MAIL_STATE': 'IL',
 'MAIL_ZIP': '62702',
 'HOST_EST_KEY': 'HOST_EST_KEY_VALUE',
 'NR_IN_ESTAB': '40',
 'OPEN_DATE': '08-FEB-18',
 'CASE_MOD_DATE': '03-DEC-23',
 'CLOSE_CONF_DATE': '08-FEB-18',
 'CLOSE_CASE_DATE': '25-FEB-19',
 'LOAD_DT': '04-DEC-23',
 'RNUM': '1'}

In [5]:
#create dataframe
df_inspections = pd.DataFrame(data)
df_inspections.head()

ACTIVITY_NR REPORTING_ID STATE_FLAG                            ESTAB_NAME  \
0   342934890      0524500             SPRINGFIELD FARM & HOME SUPPLY, INC.   
1   342965498      0524500                         FRIENDLY CHEVROLET, INC.   
2   342997244      0524500                      AMERICAN MULTI-CINEMA, INC.   
3   343056065      0524500              WEBER BUILDERS AND RENOVATORS, INC.   
4   343056156      0524500                                     HUGO ESTRADA   

                  SITE_ADDRESS    SITE_CITY SITE_STATE SITE_ZIP OWNER_TYPE  \
0         2804 N. DIRKSEN PKWY  SPRINGFIELD         IL    62702          A   
1  2450 PRAIRIE CROSSING DRIVE  SPRINGFIELD         IL    62711          A   
2        3141 MERCANTILE DRIVE  SPRINGFIELD         IL    62704          A   
3               1308 S 2ND ST.  SPRINGFIELD         IL    62704          A   
4        3000 SOUTH GRAND AVE.  SPRINGFIELD         IL    62703          A   

  OWNER_CODE  ... MAIL_STATE MAIL_ZIP        HOST_EST_KEY NR_IN_ESTAB  \
0             ...         IL    62702  HOST_EST_KEY_VALUE          40   
1             ...         IL    62711  HOST_EST_KEY_VALUE          90   
2             ...         IL    62704  HOST_EST_KEY_VALUE          24   
3             ...         IL    62703  HOST_EST_KEY_VALUE           4   
4             ...         TX    77531  HOST_EST_KEY_VALUE           4   

   OPEN_DATE CASE_MOD_DATE CLOSE_CONF_DATE CLOSE_CASE_DATE    LOAD_DT RNUM  
0  08-FEB-18     03-DEC-23       08-FEB-18       25-FEB-19  04-DEC-23    1  
1  22-FEB-18     03-DEC-23       22-FEB-18       16-MAY-19  04-DEC-23    2  
2  07-MAR-18     03-DEC-23       07-MAR-18       16-APR-18  04-DEC-23    3  
3  30-MAR-18     03-DEC-23       30-MAR-18       23-DEC-19  04-DEC-23    4  
4  30-MAR-18     03-DEC-23       30-MAR-18       27-APR-22  04-DEC-23    5  

[5 rows x 37 columns]

In [6]:
# count records. If <200 then don't need another request
len(df_inspections)

187

In [7]:
df_inspections.dtypes

ACTIVITY_NR        object
REPORTING_ID       object
STATE_FLAG         object
ESTAB_NAME         object
SITE_ADDRESS       object
SITE_CITY          object
SITE_STATE         object
SITE_ZIP           object
OWNER_TYPE         object
OWNER_CODE         object
ADV_NOTICE         object
SAFETY_HLTH        object
SIC_CODE           object
NAICS_CODE         object
INSP_TYPE          object
INSP_SCOPE         object
WHY_NO_INSP        object
UNION_STATUS       object
SAFETY_MANUF       object
SAFETY_CONST       object
SAFETY_MARIT       object
HEALTH_MANUF       object
HEALTH_CONST       object
HEALTH_MARIT       object
MIGRANT            object
MAIL_STREET        object
MAIL_CITY          object
MAIL_STATE         object
MAIL_ZIP           object
HOST_EST_KEY       object
NR_IN_ESTAB        object
OPEN_DATE          object
CASE_MOD_DATE      object
CLOSE_CONF_DATE    object
CLOSE_CASE_DATE    object
LOAD_DT            object
RNUM               object
dtype: object

In [8]:
#convert ACTIVITY_NR to integer to facilitate matching
# df_inspections['ACTIVITY_NR'] = df_inspections['ACTIVITY_NR'].astype(int)

# 3) Request and Review OSHA Violations Data

In [9]:
url = "https://data.dol.gov/get/violation"
response = requests.get(url, headers=my_key)
data = response.json()
response

<Response [200]>

In [10]:
#look at first record
data[0]

{'ACTIVITY_NR': '308258078',
 'CITATION_ID': '01002',
 'DELETE_FLAG': 'X',
 'STANDARD': '19100038 B05',
 'VIOL_TYPE': 'S',
 'ISSUANCE_DATE': '23-MAY-05',
 'ABATE_DATE': '24-SEP-08',
 'ABATE_COMPLETE': 'X',
 'CURRENT_PENALTY': '',
 'INITIAL_PENALTY': '',
 'CONTEST_DATE': '15-JUN-05',
 'FINAL_ORDER_DATE': '22-OCT-07',
 'NR_INSTANCES': '1',
 'NR_EXPOSED': '1180',
 'REC': '',
 'GRAVITY': '05',
 'EMPHASIS': '',
 'HAZCAT': '',
 'FTA_INSP_NR': '',
 'FTA_ISSUANCE_DATE': '',
 'FTA_PENALTY': '',
 'FTA_CONTEST_DATE': '',
 'FTA_FINAL_ORDER_DATE': '',
 'HAZSUB1': '',
 'HAZSUB2': '',
 'HAZSUB3': '',
 'HAZSUB4': '',
 'HAZSUB5': '',
 'LOAD_DT': '14-OCT-11',
 'RNUM': '1'}

In [11]:
#create dataframe
df_violations = pd.DataFrame(data)
df_violations.head()

ACTIVITY_NR CITATION_ID DELETE_FLAG           STANDARD VIOL_TYPE  \
0   308258078       01002           X       19100038 B05         S   
1   308258086      01001A                   19100106 B06         S   
2   308258086      01001B              19100106 E09  III         S   
3   308258086       02001                   19100038 B01         O   
4   308258086       02002                   19101200 E01         O   

  ISSUANCE_DATE ABATE_DATE ABATE_COMPLETE CURRENT_PENALTY INITIAL_PENALTY  \
0     23-MAY-05  24-SEP-08              X                                   
1     24-MAY-05  12-JUL-05              X            1000            1000   
2     24-MAY-05  12-JUL-05              X                                   
3     24-MAY-05  27-MAY-05              X                                   
4     24-MAY-05  12-JUL-05              X                                   

   ... FTA_PENALTY FTA_CONTEST_DATE FTA_FINAL_ORDER_DATE HAZSUB1 HAZSUB2  \
0  ...                                                                     
1  ...                                                                     
2  ...                                                                     
3  ...                                                                     
4  ...                                                                     

  HAZSUB3 HAZSUB4 HAZSUB5    LOAD_DT RNUM  
0                          14-OCT-11    1  
1                          14-OCT-11    2  
2                          14-OCT-11    3  
3                          14-OCT-11    4  
4                          14-OCT-11    5  

[5 rows x 30 columns]

# 4) Request Violation Data Matching Inspections

In [12]:
api_url = "https://data.dol.gov/get/violation/filter_column/ACTIVITY_NR=342934890"
response = requests.get(api_url, headers=my_key)
data = response.json()
response

<Response [200]>

In [13]:
data

[{'ACTIVITY_NR': '342934890',
  'CITATION_ID': '01001A',
  'DELETE_FLAG': '',
  'STANDARD': '19100029 B02 I',
  'VIOL_TYPE': 'S',
  'ISSUANCE_DATE': '09-MAR-18',
  'ABATE_DATE': '26-MAR-18',
  'ABATE_COMPLETE': '2',
  'CURRENT_PENALTY': '3000',
  'INITIAL_PENALTY': '6467',
  'CONTEST_DATE': '',
  'FINAL_ORDER_DATE': '26-MAR-18',
  'NR_INSTANCES': '1',
  'NR_EXPOSED': '2',
  'REC': '',
  'GRAVITY': '5',
  'EMPHASIS': '',
  'HAZCAT': '',
  'FTA_INSP_NR': '',
  'FTA_ISSUANCE_DATE': '',
  'FTA_PENALTY': '',
  'FTA_CONTEST_DATE': '',
  'FTA_FINAL_ORDER_DATE': '',
  'HAZSUB1': '',
  'HAZSUB2': '',
  'HAZSUB3': '',
  'HAZSUB4': '',
  'HAZSUB5': '',
  'LOAD_DT': '04-DEC-23',
  'RNUM': '1'},
 {'ACTIVITY_NR': '342934890',
  'CITATION_ID': '01001B',
  'DELETE_FLAG': '',
  'STANDARD': '19260451 C02 V',
  'VIOL_TYPE': 'S',
  'ISSUANCE_DATE': '09-MAR-18',
  'ABATE_DATE': '26-MAR-18',
  'ABATE_COMPLETE': '2',
  'CURRENT_PENALTY': '0',
  'INITIAL_PENALTY': '0',
  'CONTEST_DATE': '',
  'FINAL_ORDER_DAT

In [26]:
df_linked_violations = pd.DataFrame()

In [29]:
#     api_url = f"https://data.dol.gov/get/.violation/filter_column/ACTIVITY_NR={row['ACTIVITY_NR']}"
    
for index, row in df_inspections.iterrows():
    print(row['ACTIVITY_NR'])
    api_url = f"https://data.dol.gov/get/violation/filter_column/ACTIVITY_NR={row['ACTIVITY_NR']}"
    response = requests.get(api_url, headers=my_key)  # Make the request

    if response.status_code == 200:
        try:
            df_next = pd.json_normalize(response.json())
            df_linked_violations = pd.concat([df_linked_violations, df_next], ignore_index=True)
        except ValueError:
            print('Decoding JSON has failed')
    else:
        print(f"Error {response.status_code}: {response.text}")

342934890
342965498
342997244
343056065
343056156
343055018
Error 400: {
    "status": "",
    "message": "No data available, check request"
}
343099636
343097499
Error 400: {
    "status": "",
    "message": "No data available, check request"
}
343096996
Error 400: {
    "status": "",
    "message": "No data available, check request"
}
343125514
Error 400: {
    "status": "",
    "message": "No data available, check request"
}
343136396
Error 400: {
    "status": "",
    "message": "No data available, check request"
}
343214144
Error 400: {
    "status": "",
    "message": "No data available, check request"
}
343214516
Error 400: {
    "status": "",
    "message": "No data available, check request"
}
343299707
343299368
Error 400: {
    "status": "",
    "message": "No data available, check request"
}
343308557
343308672
343327730
Error 400: {
    "status": "",
    "message": "No data available, check request"
}
343362034
Error 400: {
    "status": "",
    "message": "No data availabl

345833925
345886998
Error 400: {
    "status": "",
    "message": "No data available, check request"
}
345903173
Error 400: {
    "status": "",
    "message": "No data available, check request"
}
345901227
345914071
345934863
Error 400: {
    "status": "",
    "message": "No data available, check request"
}
345951750
Error 400: {
    "status": "",
    "message": "No data available, check request"
}
345951909
345972525
345975221
Error 400: {
    "status": "",
    "message": "No data available, check request"
}
345996524
345996599
345997738
Error 400: {
    "status": "",
    "message": "No data available, check request"
}
346026545
Error 400: {
    "status": "",
    "message": "No data available, check request"
}
346050933
346486400
Error 400: {
    "status": "",
    "message": "No data available, check request"
}
346080633
Error 400: {
    "status": "",
    "message": "No data available, check request"
}
346083249
Error 400: {
    "status": "",
    "message": "No data available, check r

In [30]:
df_linked_violations.head()

ACTIVITY_NR CITATION_ID DELETE_FLAG         STANDARD VIOL_TYPE  \
0   342934890      01001A               19100029 B02 I         S   
1   342934890      01001B               19260451 C02 V         S   
2   342934890      01002A              19100178 L02 II         S   
3   342934890      01002B                 19100178 L06         S   
4   342965498       01001               19100147 F01 V         S   

  ISSUANCE_DATE ABATE_DATE ABATE_COMPLETE CURRENT_PENALTY INITIAL_PENALTY  \
0     09-MAR-18  26-MAR-18              2            3000            6467   
1     09-MAR-18  26-MAR-18              2               0               0   
2     09-MAR-18  26-MAR-18              2            2300            5174   
3     09-MAR-18  26-MAR-18              2               0               0   
4     24-MAY-18  25-JUN-18              2            5500            9054   

   ... FTA_PENALTY FTA_CONTEST_DATE FTA_FINAL_ORDER_DATE HAZSUB1 HAZSUB2  \
0  ...                                                                     
1  ...                                                                     
2  ...                                                                     
3  ...                                                                     
4  ...                                                                     

  HAZSUB3 HAZSUB4 HAZSUB5    LOAD_DT RNUM  
0                          04-DEC-23    1  
1                          04-DEC-23    2  
2                          04-DEC-23    3  
3                          04-DEC-23    4  
4                          04-DEC-23    1  

[5 rows x 30 columns]

# 5) Save Datasets

In [33]:
df_inspections.to_csv("../data/springfield-il-inspections-2018-2022.csv")

In [32]:
df_linked_violations.to_csv("../data/springfield-il-violations-2018-2022.csv")